In [1]:
def composites(wave_ds,precip_ds):    
    wavedict = {}
    
    time_phase = wave_ds["time"].astype("datetime64[D]")
    time_precip = precip_ds["time"].astype("datetime64[D]")

    phases = wave_ds.groupby("phase").groups
    # print(phases)
    for phase in phases:
        print(phase)
        phase_indices = np.where(phase == wave_ds["phase"])[0]
        
        precip_values = [
            precip_ds.where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        precip_time = [
            precip_ds["time"].where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        wavedict[str(int(phase))] = [precip_time, precip_values]
#         wavedict["time-"+str(int(phase))] = precip_time
    return(wavedict)

def composites_date(wave_ds,precip_ds):    
    wavedict = {}
    
    time_phase = wave_ds["time"].astype("datetime64[D]")
    time_precip = precip_ds["time"].astype("datetime64[D]")

    phases = wave_ds.groupby("phase").groups
    # print(phases)
    for phase in phases:
        print(phase)
        phase_indices = np.where(phase == wave_ds["phase"])[0]
        
        precip_time = [
            precip_ds["time"].where(time_phase[i] == time_precip).dropna(dim = "time").values[0]
            for i in phase_indices
        ]
        wavedict[str(int(phase))] = precip_time
#         wavedict["time-"+str(int(phase))] = precip_time
    return(wavedict)
    
def is_seas(month, seas):
    if seas=='jja':
        return (month >= 6) & (month <= 8)
    elif seas=='djf':
        return (month >= 12) | (month <= 2)
    elif seas=='son':
        return (month >= 9) & (month <= 11)
    elif seas=='mam':
        return (month >= 3) & (month <= 5)
        
def placehold(array,fillval):
    if len(array)==0:
        return(fillval)
    else: return(array)

In [2]:
import xarray as xr
import pandas as pd
import numpy as np

seas = 'mam'
if seas=='mam':
    th = 17.06493
    seasname = "SHAutumn"
elif seas=='djf':
    th = 25.44985
    seasname = "SHSummer"

In [3]:
diri = "/Users/fadhlilrizki/Documents/PhD/thesis/dataout/local_wave_phase/"
diro = "/Users/fadhlilrizki/Documents/PhD/thesis/dataout/local_wave_phase/each/"

In [4]:
regions = ["TopEnd","CapeYork","Kimberley","Queensland", "EastCoast"]
latSs   = [-17, -16, -20, -25, -20]
latNs   = [-10, -10, -13.5, -18, -16]
lonRs   = [137, 146, 128, 147, 147.5]
lonLs   = [129, 142, 120, 140, 145.5]

code = 0 #regioncode
if code==0:
    region ="TE"
elif code==1:
    region="CY"
elif code==2:
    region="KY"
elif code==3:
    region="QL"
elif code==4:
    region="EC"

In [5]:
fw_1 = "local_er_phase_"+seasname+".133.nc"
fw_2 = "local_kelvin_phase_"+seasname+".133.nc"
fw_3 = "local_mrg_phase_"+seasname+".133.nc"
fw_4 = "local_td_phase_"+seasname+".133.nc"
fw_5 = "local_mjo_phase_"+seasname+".133.nc"

In [6]:
dw_1 = xr.open_dataset(diri+fw_1)
dw_2 = xr.open_dataset(diri+fw_2)
dw_3 = xr.open_dataset(diri+fw_3)
dw_4 = xr.open_dataset(diri+fw_4)
dw_5 = xr.open_dataset(diri+fw_5)

dprec = xr.open_dataset("/Users/fadhlilrizki/Documents/PhD/thesis/data/agcd_v1_precip_calib_r025_daily_1981-2020.masked.nc")

In [7]:
dw_1 = dw_1.sel(time=slice("1981-01-01","2018-12-31"))
dw_2 = dw_2.sel(time=slice("1981-01-01","2018-12-31"))
dw_3 = dw_3.sel(time=slice("1981-01-01","2018-12-31"))
dw_4 = dw_4.sel(time=slice("1981-01-01","2018-12-31"))
dw_5 = dw_5.sel(time=slice("1981-01-01","2018-12-31"))

# dprec = dprec.sel(time=slice("1981-01-01","2018-12-31"),
#                   lon=slice(lonLs[code],lonRs[code]), 
#                   lat=slice(latSs[code],latNs[code])
#                  )

# prec = dprec.precip
# prec

In [8]:
er_active     = xr.where(dw_1["amp"] >= 1, dw_1, np.nan).dropna(dim="time")
kelvin_active = xr.where(dw_2["amp"] >= 1, dw_2, np.nan).dropna(dim="time")
mrg_active    = xr.where(dw_3["amp"] >= 1, dw_3, np.nan).dropna(dim="time")
td_active     = xr.where(dw_4["amp"] >= 1, dw_4, np.nan).dropna(dim="time")
mjo_active    = xr.where(dw_5["amp"] >= 1, dw_5, np.nan).dropna(dim="time")


In [9]:
# er_active.to_dataframe().to_csv('er_act.csv')

In [28]:
tc   = pd.read_csv('/Users/fadhlilrizki/Documents/PhD/thesis/data/TC_dates.csv',
                   usecols=["0"], parse_dates=["0"])

tc_da = xr.DataArray(tc)
er_active         = xr.where(~er_active['time'].isin(tc_da), er_active, np.nan).dropna(dim='time')
mrg_active        = xr.where(~mrg_active['time'].isin(tc_da), mrg_active, np.nan).dropna(dim='time')
kelvin_active     = xr.where(~kelvin_active['time'].isin(tc_da), kelvin_active, np.nan).dropna(dim='time')
td_active         = xr.where(~td_active['time'].isin(tc_da), td_active, np.nan).dropna(dim='time')
mjo_active        = xr.where(~mjo_active['time'].isin(tc_da), mjo_active, np.nan).dropna(dim='time')

er_active

<xarray.Dataset>
Dimensions:  (time: 2080)
Coordinates:
  * time     (time) datetime64[ns] 1981-03-01 1981-03-02 ... 2018-05-09
Data variables:
    x        (time) float32 0.7357 0.9528 1.047 ... 0.7224 0.3687 -0.02076
    y        (time) float32 0.7333 0.4502 0.0409 ... -0.8386 -1.073 -1.031
    amp      (time) float32 1.039 1.054 1.048 1.077 ... 1.029 1.107 1.134 1.032
    phase    (time) float64 8.0 8.0 1.0 2.0 2.0 3.0 ... 8.0 1.0 2.0 2.0 3.0 3.0

In [31]:
er_active         = xr.where(~er_active['time'].isin(mrg_active['time']), er_active, np.nan).dropna(dim='time')
er_active         = xr.where(~er_active['time'].isin(td_active['time']), er_active, np.nan).dropna(dim='time')
er_active         = xr.where(~er_active['time'].isin(mjo_active['time']), er_active, np.nan).dropna(dim='time')

er_active

<xarray.Dataset>
Dimensions:  (time: 106)
Coordinates:
  * time     (time) datetime64[ns] 1983-05-07 1983-05-08 ... 2018-04-26
Data variables:
    x        (time) float32 1.679 1.747 -0.6908 -0.9809 ... 2.03 2.027 1.727
    y        (time) float32 0.4131 -0.0007719 -0.872 ... 0.5065 -0.4373 -1.125
    amp      (time) float32 1.729 1.747 1.112 1.198 ... 2.179 2.092 2.074 2.061
    phase    (time) float64 1.0 1.0 4.0 4.0 3.0 4.0 ... 3.0 6.0 7.0 1.0 1.0 2.0

In [11]:
er_size = er_active.groupby("phase").groups
key_to_value_lengths = {k:len(v) for k, v in er_size.items()}
key_to_value_lengths

{1.0: 274,
 2.0: 281,
 3.0: 258,
 4.0: 244,
 5.0: 282,
 6.0: 259,
 7.0: 238,
 8.0: 244}

In [12]:
er_comp  = {}
klv_comp = {}
mrg_comp = {}
td_comp  = {}
mjo_comp = {}

In [13]:
er_phase     = pd.DataFrame(index=er_active['time'])
kelvin_phase = pd.DataFrame(index=kelvin_active['time'])
mrg_phase    = pd.DataFrame(index=mrg_active['time'])
td_phase     = pd.DataFrame(index=td_active['time'])
mjo_phase    = pd.DataFrame(index=mjo_active['time'])

In [14]:
for i in er_active.groupby('phase').groups:
    print(i)
    er = xr.where(er_active['phase']==i, er_active, np.nan).dropna(dim='time')
    er_phase[i] = er['time'].to_dataframe()
    er_phase[i] = pd.to_datetime(er_phase[i]).dt.strftime("%Y%m%d")
    del(er)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0


In [15]:
for i in kelvin_active.groupby('phase').groups:
    print(i)
    kelvin = xr.where(kelvin_active['phase']==i, kelvin_active, np.nan).dropna(dim='time')
    kelvin_phase[i] = kelvin['time'].to_dataframe()
    kelvin_phase[i] = pd.to_datetime(kelvin_phase[i]).dt.strftime("%Y%m%d")
    del(kelvin)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0


In [16]:
for i in mrg_active.groupby('phase').groups:
    print(i)
    mrg = xr.where(mrg_active['phase']==i, mrg_active, np.nan).dropna(dim='time')
    mrg_phase[i] = mrg['time'].to_dataframe()
    mrg_phase[i] = pd.to_datetime(mrg_phase[i]).dt.strftime("%Y%m%d")
    del(mrg)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0


In [17]:
for i in td_active.groupby('phase').groups:
    print(i)
    td = xr.where(td_active['phase']==i, td_active, np.nan).dropna(dim='time')
    td_phase[i] = td['time'].to_dataframe()
    td_phase[i] = pd.to_datetime(td_phase[i]).dt.strftime("%Y%m%d")
    del(td)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0


In [18]:
for i in mjo_active.groupby('phase').groups:
    print(i)
    mjo = xr.where(mjo_active['phase']==i, mjo_active, np.nan).dropna(dim='time')
    mjo_phase[i] = mjo['time'].to_dataframe()
    mjo_phase[i] = pd.to_datetime(mjo_phase[i]).dt.strftime("%Y%m%d")
    del(mjo)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0


In [19]:
er_phase.columns

Index([1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')

In [20]:
[er_phase[phase].dropna().to_csv(diro+region+"/"+region+".er"+str(phase)+"."+seas+".csv",index=False, header=False) for phase in er_phase.columns]

[None, None, None, None, None, None, None, None]

In [21]:
[kelvin_phase[phase].dropna().to_csv(diro+region+"/"+region+".kelvin"+str(phase)+"."+seas+".csv",index=False, header=False) for phase in kelvin_phase.columns]

[None, None, None, None, None, None, None, None]

In [22]:
[mrg_phase[phase].dropna().to_csv(diro+region+"/"+region+".mrg"+str(phase)+"."+seas+".csv",index=False, header=False) for phase in mrg_phase.columns]

[None, None, None, None, None, None, None, None]

In [23]:
[td_phase[phase].dropna().to_csv(diro+region+"/"+region+".td"+str(phase)+"."+seas+".csv",index=False, header=False) for phase in td_phase.columns]

[None, None, None, None, None, None, None, None]

In [24]:
[mjo_phase[phase].dropna().to_csv(diro+region+"/"+region+".mjo"+str(phase)+"."+seas+".csv",index=False, header=False) for phase in mjo_phase.columns]

[None, None, None, None, None, None, None, None]

In [25]:
! ncl 'seas="{seas}"' code={code} area_wave_extremes_perphase.ncl

 Copyright (C) 1995-2019 - All Rights Reserved
 University Corporation for Atmospheric Research
 NCAR Command Language Version 6.6.2
 The use of this software is governed by a License Agreement.
 See http://www.ncl.ucar.edu/ for more details.

Variable: pr_aave
Type: float
Total Size: 55516 bytes
            13879 values
Number of Dimensions: 1
Dimensions and sizes:	[time | 13879]
Coordinates: 
            time: [47847.375..61725.375]
Number Of Attributes: 13
  _FillValue :	1e+20
  coverage_content_type :	physicalMeasurement
  length_scale_for_analysis :	  80
  frequency :	daily
  number_of_stations_reporting :	6600
  analysis_version_number :	3.01
  cell_methods :	time: sum
  least_significant_digit :	3
  units :	mm
  long_name :	Daily precipitation
  standard_name :	lwe_thickness_of_precipitation_amount
  missing_value :	1e+20
  wgt_areaave_op_ncl :	Area Average


Variable: pr_stat
Type: float
Total Size: 120 bytes
            30 values
Number of Dimensions: 1
Dimensions and sizes:	[

In [26]:
# ! ncl 'seas="mam"' code=0 composite_combined_area_wave_extremes_forpy.ncl